In [1]:
# load your libraries
import numpy as np
import pandas as pd
import os

/Users/jessiekb/opt/anaconda3/envs/yasa_seals/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# pull in the thresholds and species codes we'll be using here
thresh = pd.read_csv('../data/thresholds_95_precision_full_test_set_with_label_weights_standard.csv')

# critically, there are 216 columns in the predictions, but the first relates to bird1.
# otherwise, the labels and this file are in the same order. make up some values
# for bird1 and plug them in here. create a new row first
new_row = pd.DataFrame({'species_code': ['bird1'], 'precision': [0.95], 'recall': [0.5], 'threshold': [0.95]})

# insert as first row
thresh = pd.concat([thresh.iloc[:0], new_row, thresh.iloc[0:]]).reset_index(drop=True)

In [7]:
# define all the raw prediction output files from SWAMP
inputDir = '../data/umich_swamp_aru_anaylsis/msid/predictions'
theFiles = os.listdir(inputDir)

# subset to npz files only
theFiles = [s for s in theFiles if 'npz' in s]

In [39]:
# loop over all the files and save out the acoustic checklists
for file in range(0, len(theFiles)):
    singleFile = os.path.join(inputDir, theFiles[file])
    data = np.load(singleFile)

    # pull out the probs
    probs = data['probs']

    # grab the ARU asset ID info
    asset_id = data['asset_ids'][0]

    # loop over the species and determine whether a species 
    # is present or not at a given threshold. create a blank species
    # vector here
    speciesList = []

    # loop over species here
    for sp in range(0,thresh.shape[0]):
        # pull the correct threshold here
        #threshold = thresh['threshold'][sp]
        threshold = 0.95
        
        if np.max(probs[:,sp]) > threshold:
            speciesList.append(thresh['species_code'][sp])
        else:
            0

    # save out the species list as a csv and give it a name corresponding to the ARU
    outputDir = '/Volumes/eBirdData/swamp_aru_dataset/acoustic_checklists'
    temp_filename = os.path.join(outputDir, asset_id + '.' + 'csv')
    
    spResults = pd.DataFrame(speciesList)
    spResults.to_csv(temp_filename, index=False)

    # close the data file
    data.close()